## Remove the extra files from BCIT Baseline Driving

This script assumes that you have copied the BaselineDrivingWorking directory into
the BaselineDrivingWorkingPhaseTwo directory.

This script removes `_events.tsv` and `_eventstemp.tsv` and then
renames `_eventstemp1.tsv` to `_events.tsv`.

In [1]:
import os
import datetime
from hed.tools import BidsTabularDictionary, get_file_list, HedLogger

# Variables to set for the specific dataset
bids_root_path = '/XXX/bcit/BaselineDrivingWorkingPhaseTwo'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
log_name = 'bcit_baseline_driving_03_remove_files_log'

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Make sure that there actually are temp_files to move
temp_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_eventstemp", exclude_dirs=exclude_dirs)
if not temp_files:
    raise Exception("There are no eventstemp files to move to events")

# Construct the event file dictionaries for the BIDS and for EEG.event files
files_bids = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events", exclude_dirs=exclude_dirs)
bids_dict = BidsTabularDictionary("Bids event files", files_bids, entities=entities)

# Remove the appropriate files
for key, file in bids_dict.iter_files():
    # Remove events.tsv
    filename = file.file_path
    os.remove(filename)
    logger.add(key, f"Remove {os.path.basename(filename)}")

    # Remove eventstemp.tsv
    filename_temp = filename[:-4] + "temp.tsv"
    if os.path.isfile(filename_temp):
        os.remove(filename_temp)
        logger.add(key, f"Removed {os.path.basename(filename_temp)}")
    else:
        logger.add(key, f"{os.path.basename(filename_temp)} does not exist and could not be removed", level="ERROR", also_print=True)

    # Move eventstemp1.tsv to events.tsv
    filename_temp1 = filename[:-4] + "temp1.tsv"
    if os.path.isfile(filename_temp1):
        os.rename(filename_temp1, filename)
        logger.add(key, f"Renamed {os.path.basename(filename_temp1)}\n\t\tto {os.path.basename(filename)}")
    else:
        logger.add(key, f"{os.path.basename(filename_temp1)} does not exist and could not be moved", level="ERROR", also_print=True)

# Now make sure there aren't any unexpected leftovers
temp_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_eventstemp", exclude_dirs=exclude_dirs)
if temp_files:
    for file in temp_files:
        logger.add('', f"{os.path.basename(file)} is unexpectedly left over", level="ERROR", also_print=True)

temp1_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_eventstemp1",
                           exclude_dirs=exclude_dirs)
if temp1_files:
    for file in temp1_files:
        logger.add('', f"{os.path.basename(file)} is unexpectedly left over", level="ERROR", also_print=True)

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)

save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
bcit_baseline_driving_03_remove_files_log: Level None
sub-01_ses-01_run-1:
	[ Remove sub-01_ses-01_task-DriveWithSpeedChange_run-1_events.tsv]
	[ Removed sub-01_ses-01_task-DriveWithSpeedChange_run-1_eventstemp.tsv]
	[ Renamed sub-01_ses-01_task-DriveWithSpeedChange_run-1_eventstemp1.tsv
		to sub-01_ses-01_task-DriveWithSpeedChange_run-1_events.tsv]
sub-02_ses-01_run-1:
	[ Remove sub-02_ses-01_task-DriveWithSpeedChange_run-1_events.tsv]
	[ Removed sub-02_ses-01_task-DriveWithSpeedChange_run-1_eventstemp.tsv]
	[ Renamed sub-02_ses-01_task-DriveWithSpeedChange_run-1_eventstemp1.tsv
		to sub-02_ses-01_task-DriveWithSpeedChange_run-1_events.tsv]
sub-03_ses-01_run-1:
	[ Remove sub-03_ses-01_task-DriveWithSpeedChange_run-1_events.tsv]
	[ Removed sub-03_ses-01_task-DriveWithSpeedChange_run-1_eventstemp.tsv]
	[ Renamed sub-03_ses-01_task-DriveWithSpeedChange_run-1_eventstemp1.tsv
		to sub-03_ses-01_task-DriveWithSpeedChange_run-1_events.tsv]
sub-04_ses-01_run-1:
	[ Remove sub-04_